In [1]:
!git clone https://github.com/gmonteiroc21/desafio_bookbridge.git

Cloning into 'desafio_bookbridge'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 56 (delta 20), reused 7 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 23.51 KiB | 3.92 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [2]:
%cd desafio_bookbridge

/content/desafio_bookbridge


###1- Importação e Limpeza

In [3]:
#importação do dataset seguindo as instruções na pagina

import kagglehub

# Download latest version
data_path = kagglehub.dataset_download("pypiahmad/goodreads-book-reviews1")

print("Path to dataset files:", data_path)

100%|██████████| 8.14G/8.14G [01:35<00:00, 91.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1


In [36]:
import pandas as pd
import json
import os
import sys
sys.path.append(os.path.abspath("scripts"))
from scripts.manipulating_data import *
from scripts.data_analysis import *
from scripts.data_preparation import *
from scripts.recommender import *

file_path = "/root/.cache/kagglehub/datasets/pypiahmad/goodreads-book-reviews1/versions/1/"


NameError: name 'user_ranking_df' is not defined

In [4]:
#transforma as interações dos usuarios em um dataframe
interactions_df = pd.read_csv(file_path+'goodreads_interactions.csv', nrows=20000000) #limita em  2M de linhas
print(interactions_df.head(1))

   user_id  book_id  is_read  rating  is_reviewed
0        0      948        1       5            0


In [12]:
#transforma as informações dos livros em um dataframe
df = pd.read_json(file_path+'goodreads_books.json', lines=True, nrows=100000) #limita em 100mil linhas
colunas_desejadas = ['book_id',  'authors' ,'title', 'average_rating', 'ratings_count',
              'num_pages', 'publication_year', 'text_reviews_count']
books_df = df[colunas_desejadas]


tratamento do *books_df*

In [13]:
first_df = clean_empty_rows(books_df, 'book_id')

'''books_df = duplicate_size(file_path+'goodreads_books.json', books_df, colunas_desejadas)'''

books_df['average_rating'] = pd.to_numeric(books_df['average_rating'], errors='coerce')

print(books_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   book_id             100000 non-null  int64  
 1   authors             100000 non-null  object 
 2   title               100000 non-null  object 
 3   average_rating      99981 non-null   float64
 4   ratings_count       100000 non-null  object 
 5   num_pages           100000 non-null  object 
 6   publication_year    100000 non-null  object 
 7   text_reviews_count  100000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 6.1+ MB
None


<ipython-input-13-21e8220980cd>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_df['average_rating'] = pd.to_numeric(books_df['average_rating'], errors='coerce')


In [14]:
#transforma as informações dos autores em um dataframe
authors_df = pd.read_json(file_path+'goodreads_book_authors.json', lines=True)
print(authors_df.head(1))

   average_rating  author_id  text_reviews_count              name  \
0            3.98     604031                   7  Ronald J. Fields   

   ratings_count  
0             49  


In [5]:
#transforma as informações de generos em um dataframe
genres_df = pd.read_json(file_path+'goodreads_book_genres_initial.json', lines=True)
print(genres_df.head(10))


    book_id                                             genres
0   5333265      {'history, historical fiction, biography': 1}
1   1333909  {'fiction': 219, 'history, historical fiction,...
2   7327624  {'fantasy, paranormal': 31, 'fiction': 8, 'mys...
3   6066819  {'fiction': 555, 'romance': 23, 'mystery, thri...
4    287140                                 {'non-fiction': 3}
5    287141  {'history, historical fiction, biography': 9, ...
6    378460                                     {'fiction': 2}
7   6066812  {'fantasy, paranormal': 32, 'young-adult': 8, ...
8  34883016                                     {'romance': 3}
9    287149  {'non-fiction': 24, 'history, historical ficti...


In [37]:
books_df = books_df.merge(
    genres_df[['book_id', 'genres']],  # Seleciona apenas 'book_id' e 'genres'
    on='book_id',  # A chave de junção
    how='left'  # Mantém todos os livros, mesmo aqueles sem gênero
)

### 2 - Análise exploratória e preparação dos dados para modelagem

In [9]:
unique_genres = ['history, historical fiction, biography', 'fiction', 'fantasy, paranormal'
 ,'mystery, thriller, crime', 'poetry', 'romance', 'non-fiction', 'children',
 'young-adult', 'comics, graphic']


In [14]:
genre_stats_df = genre_stats(genres_df, books_df, unique_genres)
print(genre_stats_df)

         book_id                                  genres  average_rating  \
86374   35559368  history, historical fiction, biography            5.00   
33599   20996579                                 fiction            5.00   
79068   31344082                     fantasy, paranormal            5.00   
221166  16127783                mystery, thriller, crime            5.00   
197341  15809673                                  poetry            5.00   
104733  35511414                                 romance            5.00   
172653  25457536                             non-fiction            4.94   
118251  29004052                                children            5.00   
169460  33670726                             young-adult            4.93   
118250  29004052                         comics, graphic            5.00   

        ratings_count                                              title  
86374              10                             Why Did You Choose Me?  
33599        

In [11]:
top_genre_per_user = user_preference(interactions_df, genres_df)
print(top_genre_per_user)

     user_id                          favorite_genre  genre_count
3          0                                 fiction          364
16         1                             non-fiction           39
25         2                             non-fiction           55
32         3                                 fiction          133
42         4                                 fiction           87
52         5                                 fiction         2080
62         6                                 fiction           54
72         7                                 fiction           49
85         8                             non-fiction           64
92         9                                 fiction          117
102       10                                 fiction           56
112       11                                 fiction          243
122       12                                 fiction           51
133       13  history, historical fiction, biography          108
142       

In [15]:
user_literary_age = calculate_user_literary_age(interactions_df, books_df)
user_literary_age

,user_id,70-,70-00,00+
0,0,0,3,6
1,1,0,0,1
2,2,0,1,2
3,3,0,2,0
4,4,0,4,4
...,...,...,...,...
32584,39219,1,2,3
32585,39220,0,2,5
32586,39221,0,3,1
32587,39222,0,2,0


In [47]:
user_id = 18
user_genre = top_genre_per_user[top_genre_per_user['user_id'] == user_id]['favorite_genre'].iloc[0]

# Filtra o DataFrame de estatísticas de gêneros para pegar a linha correspondente ao gênero
best_book_in_genre = genre_stats_df[genre_stats_df['genres'] == user_genre]

# Agora, você pode acessar as informações do melhor livro desse gênero
best_book = best_book_in_genre['title'].iloc[0]  # Pega o nome do melhor livro

# Exibe o melhor livro do gênero
print(best_book)


Dominoes: Part 3
